In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dados_vins.csv', index_col=0)

In [3]:
df_precos = df.groupby('PN').agg({'Price':'mean'})

In [4]:
df_qtd = df.groupby(['phantom', 'PN']).agg({'Qtde':lambda x: list(x)}).reset_index()

In [5]:
df_qtd_preco = pd.merge(left=df_qtd, right=df_precos, on='PN', how='left')

In [6]:
MULTA = 5000

def otimiza(qtde, price):
    espaco_busca = np.sort(np.array(qtde))

    espaco_probs = np.array([np.sum(espaco_busca > x) for x in espaco_busca]) / espaco_busca.shape[0]  # utiliza a distribuição empírica dos dados para calcular P(Y>X)
    espaco_resultado = MULTA*espaco_probs + price*(espaco_busca - np.mean(espaco_busca))*(1-espaco_probs)
    
    index_x = np.argmin(espaco_resultado)

    return {'qtd_min':espaco_busca[0], 'qtd_max':espaco_busca[-1], 'valor_recomendado':espaco_busca[index_x], 'custo_esperado':espaco_resultado[index_x]}

In [7]:
resultados = pd.DataFrame(df_qtd_preco.apply(lambda x: otimiza(x['Qtde'], x['Price']), axis=1).to_list())

In [8]:
df_recomendacoes = pd.concat([df_qtd_preco, resultados], axis=1)

In [9]:
df_recomendacoes.query('valor_recomendado!=qtd_max')

,phantom,PN,Qtde,Price,qtd_min,qtd_max,valor_recomendado,custo_esperado


In [10]:
df_recomendacoes

,phantom,PN,Qtde,Price,qtd_min,qtd_max,valor_recomendado,custo_esperado
0,3941,Parafuso 10x10 10.4,"[59, 20, 127, 88, 59, 68, 98, 215, 225, 107, 5...",1.049138,20,225,225,127.675508
1,3941,Parafuso 10x10 12.4,"[254, 98, 107, 68, 176, 78, 215, 68, 195, 234,...",1.191962,68,254,254,122.374801
2,3941,Parafuso 10x12 10.4,"[29, 137, 244, 39, 205, 215, 29, 215, 225, 176...",1.137020,29,254,254,130.398260
3,3941,Parafuso 10x12 12.4,"[137, 29, 127, 10, 78, 117, 195, 117, 186, 68,...",1.003228,10,215,215,96.023294
4,3941,Parafuso 10x14 10.4,"[147, 156, 195, 244, 254, 49, 225, 225, 225, 8...",1.060225,10,254,254,110.095952
...,...,...,...,...,...,...,...,...
427,94826,Parafuso 8x16 10.4,"[166, 147, 68, 49, 39, 127, 137, 176, 137, 10,...",1.095716,10,234,234,135.923570
428,94826,Parafuso 8x16 12.4,"[49, 78, 20, 59, 254, 205, 186, 225, 186, 88, ...",1.038933,20,254,254,122.548929
429,94826,Rebite 10,"[205, 39, 137, 117, 254, 156, 147, 88, 29, 88,...",0.857407,29,254,254,117.274175
430,94826,Rebite 12,"[195, 176, 225, 10, 244, 147, 234, 78, 117, 22...",1.048368,10,254,254,122.820393
